# Just an example.You can alter sample code anywhere.

## Mount your google drive

In [50]:
#from google.colab import drive
#drive.mount('/content/drive')

In [51]:
# You need to modify this part to the directory where your code is located
#%cd "/content/drive/MyDrive/DL_Lab1/"

## Import packages

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import time

In [53]:
#Fix the random seed
np.random.seed(0)
torch.manual_seed(0)

## Load the data and label

In [54]:
train_load = np.loadtxt('./data/fmnist-train.csv',delimiter=',',dtype="int")
train_data=train_load[:,1:]
train_label=train_load[:,0]
print("shape of train_data: {}".format(train_data.shape))
print("shape of train_label: {}".format(train_label.shape))

shape of train_data: (60000, 784)
shape of train_label: (60000,)


## Show the training data

In [55]:
# uncomment if you want to show the training data
#plt.figure(figsize=(20, 20))
#for index in range(10):
#    image = train_data[index+20000].reshape(28,28)
#    plt.subplot(2, 5, index+1)
#    plt.imshow(image)
#plt.show()

In [56]:
train_image_num = train_data.shape[0]
train_data = train_data.astype('float32')

print("train_image_num  is : {}".format(train_image_num))

train_image_num  is : 60000


## Change numpy array to pytorch tensor

In [57]:
train_data_tensor = torch.from_numpy(train_data)
train_label_tensor = torch.from_numpy(train_label)

## Validation image number

In [58]:
val_image_num=10000

## Convert labels to one hot vector


In [59]:
label_temp = np.zeros((train_image_num, 10), dtype = np.float32)
for i in range(train_image_num):
    label_temp[i][train_label[i]] = 1
train_label_onehot = np.copy(label_temp)
train_label_onehot_tensor = torch.from_numpy(train_label_onehot)
print("One-hot training labels shape:",train_label_onehot.shape)


One-hot training labels shape: (60000, 10)


## Hyperparameters

In [60]:
EPOCH = 20
Batch_size = 10000# 10000 should be divisible by batch_size
Learning_rate = 0.001

## Define the models with pytorch

In [61]:
in_channels = 1
conv_out_channels = 8
kernel_size = 3
num_classes = 10
fc_hidden_size = 128

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Define the network layers
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=conv_out_channels,
                      kernel_size=kernel_size,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Flatten()
        )

        conv_output_height = (28 + 2 * 1 - kernel_size) // 1 + 1
        conv_output_width = (28 + 2 * 1 - kernel_size) // 1 + 1
        flattened_size = conv_out_channels * conv_output_height * conv_output_width

        self.classifier = nn.Sequential(
            nn.Linear(flattened_size, fc_hidden_size),
            nn.ReLU(),
            nn.Linear(fc_hidden_size, num_classes)
        )

    def forward(self, x):
        # The input x has shape (batch_size, 784)
        x = x.view(-1, in_channels, 28, 28)
        x = self.features(x)
        x = self.classifier(x)
        return x

net = Net()

## Criterion and Optimizer

In [62]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=Learning_rate)

## Training

In [63]:
import time
train_batch_num = (train_image_num - val_image_num )//Batch_size
val_batch_num = (val_image_num)//Batch_size

for epoch in range(1, EPOCH+1):
    train_hit = 0
    val_hit = 0
    total_train_loss = 0.0
    total_val_loss = 0.0
    start_time = time.time()

    # Shuffle the training data at the beginning of each epoch
    perm = torch.randperm(train_data_tensor.size(0))
    train_data_shuffled = train_data_tensor[perm]
    train_label_shuffled = train_label_tensor[perm]
    train_label_onehot_shuffled = train_label_onehot_tensor[perm]

    for it in range(train_batch_num):
        inputs = train_data_shuffled[it*Batch_size:(it+1)*Batch_size]
        labels = train_label_shuffled[it*Batch_size:(it+1)*Batch_size]

        optimizer.zero_grad()
        outputs = net(inputs)

        _, pred_index = torch.max(outputs, 1)
        train_hit += (pred_index == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    net.eval()
    with torch.no_grad():
        for titt in range(val_batch_num):
            tit=train_batch_num+titt
            inputs = train_data_tensor[tit*Batch_size:(tit+1)*Batch_size]
            labels = train_label_tensor[tit*Batch_size:(tit+1)*Batch_size]

            outputs = net(inputs)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_hit += (predicted == labels).sum().item()
    end_time = time.time()
    epoch_time = end_time - start_time
    print('Task-2  | Epoch:%3d'%epoch, ' |Train Loss:%8.4f'%(total_train_loss/train_batch_num), ' |Train Acc:%3.4f'%(train_hit/(train_image_num-val_image_num)*100.0)
          , ' |Val Loss:%8.4f'%(total_val_loss/val_batch_num), ' |Val Acc:%3.4f'%(val_hit/val_image_num*100.0), ' |Epoch time:%5.2f'%(epoch_time),' sec')

Task-2  | Epoch:  1  |Train Loss: 71.9641  |Train Acc:22.0380  |Val Loss: 51.7723  |Val Acc:37.8500  |Epoch time: 1.76  sec
Task-2  | Epoch:  2  |Train Loss: 33.1804  |Train Acc:42.9140  |Val Loss: 14.2820  |Val Acc:48.1300  |Epoch time: 1.79  sec
Task-2  | Epoch:  3  |Train Loss:  7.6198  |Train Acc:55.4320  |Val Loss:  4.3894  |Val Acc:60.6700  |Epoch time: 1.76  sec
Task-2  | Epoch:  4  |Train Loss:  3.6268  |Train Acc:58.4960  |Val Loss:  2.3680  |Val Acc:64.5000  |Epoch time: 1.75  sec
Task-2  | Epoch:  5  |Train Loss:  2.3144  |Train Acc:66.2980  |Val Loss:  1.9523  |Val Acc:69.0500  |Epoch time: 1.74  sec
Task-2  | Epoch:  6  |Train Loss:  1.8201  |Train Acc:70.8440  |Val Loss:  1.5283  |Val Acc:70.1100  |Epoch time: 1.73  sec
Task-2  | Epoch:  7  |Train Loss:  1.4988  |Train Acc:68.2900  |Val Loss:  1.1420  |Val Acc:74.8700  |Epoch time: 1.70  sec
Task-2  | Epoch:  8  |Train Loss:  1.1426  |Train Acc:75.5540  |Val Loss:  1.0774  |Val Acc:76.1600  |Epoch time: 1.71  sec
Task-2  